In [1]:
!pip install torchinfo

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from dataloader import Dataset
from torchinfo import summary
import time
import copy
import pdb
from tqdm import tqdm

In [4]:
#training parameters in neptune format
PARAMS = {
    "input_size": 416,
    "output_size": 416,
    "model": "vgg_unet",
    "learning_rate": 0.0001,
    "batch_size": 8,
    'epochs': 1000,
    'patience': 10,
    "train_dataset_size": -1, # set train dataset subset. Useful when neet to 
                              # overtrain model with small amount of images.
                              # -1 -all images from train directories.
    "test_dataset_size": -1,  # set test dataset subset.
                              # -1 -all images from train directories.
    "n_classes": 2,
    'image_preload': False,
}

In [5]:
import configparser
config = configparser.ConfigParser()
config.read("./config.cfg")

['./config.cfg']

In [11]:
#neptune installation and initialization
#!pip install neptune-client
import neptune
neptune.init(
             project_qualified_name='moad.s.k/MSc-Project',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1MWE3YmU1ZC1jNWViLTQ1ZjUtYTE4NC1mOTgzYmUxNmI5MDkifQ==' # your credentials

             #project_qualified_name=config["neptune"]["project"],
             #api_token=config["neptune"]["token"],
             )
neptune.create_experiment(params=PARAMS)

https://app.neptune.ai/moad.s.k/MSc-Project/e/MSCPROJ-1


Experiment(MSCPROJ-1)

Unexpected error in ping thread.
Traceback (most recent call last):
  File "C:\Users\Moad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Moad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Moad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Moad\anaconda3\lib\http\client.py", line 1347, in getresponse
    response.begin()
  File "C:\Users\Moad\anaconda3\lib\http\client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Moad\anaconda3\lib\http\client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Moad\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  

In [14]:
#dataset configuration

import os

dataset_dir = os.path.normpath("sentinel-river-segmentation-dataset/")
x_train_dir = os.path.join(dataset_dir,"x_train")
y_train_dir = os.path.join(dataset_dir,"y_train")
x_test_dir = os.path.join(dataset_dir,"x_test")
y_test_dir = os.path.join(dataset_dir,"y_test")

train_set = Dataset("sentinel-river-segmentation-dataset/x_train", "sentinel-river-segmentation-dataset/y_train", input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["train_dataset_size"])
test_set = Dataset("sentinel-river-segmentation-dataset/x_test", "sentinel-river-segmentation-dataset/y_test", input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["test_dataset_size"])

batch_size = PARAMS['batch_size']
dataloaders = {
    'train': DataLoader(train_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0),
    'val': DataLoader(test_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0)
}

ValueError: num_samples should be a positive integer value, but got num_samples=0